In [6]:
import pandas as pd
import glob 


import nltk
from nltk.corpus import stopwords    
from tqdm.notebook import tqdm

import random 
import xml.etree.ElementTree as ET
import os
import glob
import random

from collections import Counter

from nltk import ngrams 
from nltk.util import everygrams

import nltk

import json

from tqdm import tqdm

In [11]:
stp_wrds = stopwords.words('english')  

# 1. Download dictionaries

* download dictionaries from [linguatools(https://linguatools.org/tools/corpora/wikipedia-parallel-titles-corpora/)
* unzip and rename files for easier use

In [ ]:
!wget https://www.dropbox.com/s/gqirltluna8z76r/wikititles-2014_deen.xml.gz 
!wget https://www.dropbox.com/s/5vns1b698wzf4v8/wikititles-2014_enit.xml.gz
!wget https://www.dropbox.com/s/58swse04brrnt9h/wikititles-2014_aren.xml.gz
!wget https://www.dropbox.com/s/jqeiybu797zj434/wikititles-2014_enru.xml.gz
!wget https://www.dropbox.com/s/1fkx4y98155b7bp/wikititles-2014_ennl.xml.gz

!mv wikititles-2014_aren.xml.gz wikititles-2014_enar.xml.gz
!mv wikititles-2014_deen.xml.gz wikititles-2014_ende.xml.gz

for  target_lang in ['de', 'ar', 'it', 'ru', 'nl']:
    !gzip -d wikititles-2014_{source_lang}{target_lang}.xml.gz
    !rm wikititles-2014_{source_lang}{target_lang}.xml.gz

# 2. Process dictionaries 
* convert from XML to {word_{en} : word_{tgt}} format
* store in [feather](https://github.com/wesm/feather) files

In [ ]:
def clean(text):
    x = text.lower()
    if ':' in x:
        x = x.split(':')[1].strip()
    if '(' in x:
        x = x.split('(')[0].strip()
    return x.strip()

def get_cs_dict(target_lang):
    tree = ET.parse(f'wikititles-2014_en{target_lang}.xml')
    root = tree.getroot()

    records = []


    for child in root:
        record = ['','']
        for grandchild in child:
            if grandchild.tag == 'entry':
                if grandchild.attrib['lang'] == target_lang:
                    record[1] = grandchild.text
                if grandchild.attrib['lang'] == 'en':
                    record[0] = grandchild.text
        records.append(record)

    cs_df = pd.DataFrame(records, columns = ['en',target_lang])

    cs_df[target_lang] = cs_df[target_lang].apply(clean)
    cs_df[target_lang] = cs_df[target_lang].str.lower()
    cs_df['en'] = cs_df['en'].str.lower()
    cs_df.to_feather(target_lang)
    
# for  target_lang in ['de', 'ar', 'it', 'ru', 'nl']:
#     get_cs_dict(target_lang)

# 3. Code-switching
* Input: text, target dictionary
* Output: code-switched text, list of replaced entities 

In [2]:
def code_switch(text, cs_dict):
    text = text.lower()
    
    tokens = text.split()
    entities = {}
    
    tokens = list(everygrams(tokens, max_len=3))
    tokens = [' '.join(token) for token in tokens]
    tokens = [token for token in tokens if token not in stp_wrds]
    tokens = sorted(tokens, key=len)[::-1]
    tokens = [token for token in tokens if token in cs_dict]
    
    for token in tokens:
        entity = cs_dict[token]
        text1 = text.replace(token, entity)
        if text1 != text:
            entities[token] = entity
        text = text1
        
    # print(entities)
        
    return text, entities

# 4. Example

In [7]:
source_lang = 'en'
target_lang = 'de'

In [8]:
print('target language is', target_lang)
print('reading dict...')
cs_df = pd.read_feather(target_lang)
print('dict is read...')
print('converting dict...')
cs_dict = dict(zip(cs_df[source_lang],cs_df[target_lang]))

target language is de
reading dict...
dict is read...
converting dict...


In [ ]:
s = 'What is an affinity credit card program?'
cs_s = code_switch(s, cs_dict)
print('before:', s)
print('after:', s)